In [9]:
import pycuda.autoinit
import pycuda
import torch
import torch.nn as nn
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
from torch.utils.cpp_extension import load
import os

In [10]:
current_path = "./qtorch/quant/"
if torch.cuda.is_available():
    quant_cuda = load(
        name="quant_cuda",
        sources=[
            os.path.join(current_path, "quant_cuda/quant_cuda.cpp"),
            os.path.join(current_path, "quant_cuda/bit_helper.cu"),
            os.path.join(current_path, "quant_cuda/sim_helper.cu"),
            os.path.join(current_path, "quant_cuda/block_kernel.cu"),
            os.path.join(current_path, "quant_cuda/float_kernel.cu"),
            os.path.join(current_path, "quant_cuda/fixed_point_kernel.cu"),
            os.path.join(current_path, "quant_cuda/quant.cu"),
        ],
    )

In [65]:
quant_cuda.f

<module 'quant_cuda' from '/home/pdh/.cache/torch_extensions/py38_cu118/quant_cuda/quant_cuda.so'>

In [37]:
a = torch.Tensor([int('0x3f111111', 0), int('0x3fc00000', 0), int('0x40800000', 0)]).type(torch.int32)

In [ ]:
quant_cuda.

In [68]:
torch.normal(mean=torch.Tensor([0]*10), std=torch.Tensor([10]*10))

tensor([ -9.4045,   9.0723,   6.3809,  -5.1546, -10.6995,  20.6390,   9.6401,
         18.1697,  -5.8983,   5.4089])

In [69]:
#a_float = a.view(torch.float32)
#print(a)
a_float = torch.randn([10])
print(a_float.abs())
a_float = a_float.abs()+ 1e-8
print("a's exponent: ", torch.log2(a_float).ceil())

tensor([0.1477, 0.9345, 0.7838, 0.2859, 0.5894, 0.2157, 0.6833, 1.4652, 0.0089,
        2.2961])
a's exponent:  tensor([-2., -0., -0., -1., -0., -2., -0.,  1., -6.,  2.])


In [75]:
for i in range(10):
    print(quant_cuda.block_quantize_stochastic(a_float.cuda(), 4, 0))

tensor([0.1562, 0.8750, 0.7500, 0.3125, 0.6250, 0.2188, 0.7500, 1.5000, 0.0098,
        2.5000], device='cuda:0')
tensor([0.1562, 0.8750, 0.7500, 0.3125, 0.5000, 0.2188, 0.7500, 1.5000, 0.0078,
        2.0000], device='cuda:0')
tensor([0.1562, 0.8750, 0.7500, 0.3125, 0.5000, 0.2188, 0.7500, 1.5000, 0.0098,
        2.0000], device='cuda:0')
tensor([0.1562, 0.8750, 0.7500, 0.3125, 0.6250, 0.1875, 0.7500, 1.5000, 0.0098,
        2.5000], device='cuda:0')
tensor([0.1562, 0.8750, 0.7500, 0.3125, 0.6250, 0.2188, 0.7500, 1.5000, 0.0098,
        2.5000], device='cuda:0')
tensor([0.1562, 0.8750, 0.8750, 0.3125, 0.6250, 0.2188, 0.6250, 1.5000, 0.0078,
        2.5000], device='cuda:0')
tensor([0.1562, 0.8750, 0.7500, 0.3125, 0.5000, 0.2188, 0.6250, 1.5000, 0.0098,
        2.0000], device='cuda:0')
tensor([0.1562, 0.8750, 0.7500, 0.2500, 0.6250, 0.2188, 0.7500, 1.5000, 0.0098,
        2.5000], device='cuda:0')
tensor([0.1562, 0.8750, 0.8750, 0.3125, 0.6250, 0.2188, 0.7500, 1.5000, 0.0098,
        

In [46]:
import torch
import cutex

M, N, K = 100, 100, 10
a = torch.rand((M, K), dtype=torch.float32).cuda() * 1/100
b = torch.rand((K, N), dtype=torch.float32).cuda() * 1/100
c = torch.empty((M, N), dtype=torch.float32).cuda()

#a = torch.randint(1, 100, (M, K), dtype=torch.float32).cuda()
#b = torch.randint(1, 100, (K, N), dtype=torch.float32).cuda()
#c = torch.randint(1, 100, (M, N), dtype=torch.float32).cuda()


kernels = cutex.SourceModule(r"""
__global__ void matmul(Tensor<float, 2> *a, Tensor<float, 2> *b, Tensor<float, 2> *c, int M, int N, int K) {
    int m = blockIdx.y * blockDim.y + threadIdx.y;
    int n = blockIdx.x * blockDim.x + threadIdx.x;
    float v = 0.f;
    if (m >= M || n >= N) return;
    for (int k = 0; k < K; ++k) {
        v += (*a)[m][k] * (*b)[k][n];
    }
    (*c)[m][n] = v;
}
""",float_bits=32, int_bits=32)

kernels.matmul(a, b, c, M, N, K, grid=(N // 32 + 1, M // 32 + 1), block=(32, 32, 1))

torch.cuda.synchronize()

if not torch.allclose(c, torch.mm(a, b)):
    print("gpu :", (c - torch.mm(a,b)).mean())
    print("cpu :", (c - torch.mm(a.cpu(), b.cpu()).cuda()).mean())
    print(a[0, 0:10])
    print(b[0:10, 0])
    print("compare 0:10 :", (c - torch.mm(a,b))[0,0:10])

gpu : tensor(-3.3881e-09, device='cuda:0')
cpu : tensor(0., device='cuda:0')
tensor([0.0065, 0.0009, 0.0003, 0.0047, 0.0097, 0.0041, 0.0070, 0.0017, 0.0052,
        0.0054], device='cuda:0')
tensor([0.0008, 0.0046, 0.0031, 0.0018, 0.0046, 0.0003, 0.0039, 0.0072, 0.0068,
        0.0039], device='cuda:0')
compare 0:10 : tensor([ 1.3053e-08,  3.0544e-08, -4.0847e-08,  1.5745e-08, -4.0309e-09,
        -2.4185e-08,  1.1642e-08, -3.3790e-08, -1.9994e-08, -1.7695e-08],
       device='cuda:0')


In [47]:
import torch
import cutex



bit_helper = cutex.SourceModule(r"""
#define FLOAT_TO_BITS(x) (*reinterpret_cast<unsigned int*>(x))
#define BITS_TO_FLOAT(x) (*reinterpret_cast<float*>(x))

__device__ __forceinline__ unsigned int extract_exponent(float *a) {
  unsigned int temp = *(reinterpret_cast<unsigned int*>(a));
  temp = (temp << 1 >> 24); // single preciision, 1 sign bit, 23 mantissa bits
  return temp-127+1; // exponent offset and virtual bit
}

__device__ __forceinline__ unsigned int round_bitwise_stochastic(unsigned int target,
                                                                 unsigned int rand_prob,
                                                                 int man_bits) {
    unsigned int mask = (1 << (23-man_bits)) - 1;
    unsigned int add_r = target+(rand_prob & mask);
    unsigned int quantized = add_r & ~mask;
    return quantized;
}

__device__ __forceinline__ unsigned int round_bitwise_nearest(unsigned int target,
                                                              int man_bits) {
    unsigned int mask = (1 << (23-man_bits)) - 1;
    unsigned int rand_prob = 1 << (23-man_bits-1);
    unsigned int add_r = target+rand_prob;
    unsigned int quantized = add_r & ~mask;
    return quantized;
}

__device__ __forceinline__ unsigned int clip_exponent(int exp_bits, int man_bits,
                                                      unsigned int old_num,
                                                      unsigned int quantized_num) {
  if (quantized_num == 0)
    return quantized_num;

  int quantized_exponent_store = quantized_num << 1 >> 1 >> 23; // 1 sign bit, 23 mantissa bits
  int max_exponent_store = (1 << (exp_bits - 1)) + 127; // we are not reserving an exponent bit for infinity, nan, etc
  // Clippping Value Up
  if (quantized_exponent_store > max_exponent_store)
  {
    unsigned int max_man = (unsigned int)-1 << 9 >> 9 >> (23 - man_bits) << (23 - man_bits); // 1 sign bit, 8 exponent bits, 1 virtual bit
    unsigned int max_num = ((unsigned int)max_exponent_store << 23) | max_man;
    unsigned int old_sign = old_num >> 31 << 31;
    quantized_num = old_sign | max_num;
  }
  return quantized_num;
}


__device__ __forceinline__ unsigned int clip_max_exponent(int man_bits,
                                                          unsigned int max_exponent,
                                                          unsigned int quantized_num) {
  unsigned int quantized_exponent = quantized_num << 1 >> 24 << 23; // 1 sign bit, 23 mantissa bits
  if (quantized_exponent > max_exponent) {
    unsigned int max_man = (unsigned int ) -1 << 9 >> 9 >> (23-man_bits) << (23-man_bits); // 1 sign bit, 8 exponent bits
    unsigned int max_num = max_exponent | max_man;
    unsigned int old_sign = quantized_num >> 31 << 31;
    quantized_num = old_sign | max_num;
  }
  return quantized_num;
}
""",float_bits=32, int_bits=32)


In [34]:
c-torch.mm(a,b)
print(c)

tensor([[2.8954, 2.2833, 2.9487,  ..., 3.2966, 1.7785, 3.2356],
        [1.6707, 1.8414, 2.6955,  ..., 2.4816, 1.2431, 2.4465],
        [2.4314, 2.6764, 3.1013,  ..., 3.1104, 1.7638, 3.6102],
        ...,
        [2.5037, 1.5539, 1.9033,  ..., 2.5582, 1.5119, 2.4344],
        [2.3465, 2.9766, 3.3992,  ..., 3.1431, 1.6484, 3.1594],
        [3.1410, 2.5632, 1.7822,  ..., 2.8549, 2.5160, 2.6298]],
       device='cuda:0')
